# Goal: characterize the multi-mapping within *example_transcriptome* using findMM.py

**info:**
- I am using bash
- bash commands are executed in this notebook with the `!` character preceding the command

`example_transcriptome.fasta` is a small portion of the yeast S288C transcriptome in `fasta` format that will be used for this demonstration

In [6]:
!head example_transcriptome.fasta

>YKL166C TPK3 SGDID:S000001649, Chr XI from 135705-134509, Genome Release 64-2-1, reverse complement, Verified ORF, "cAMP-dependent protein kinase catalytic subunit; promotes vegetative growth in response to nutrients via the Ras-cAMP signaling pathway; partially redundant with Tpk1p and Tpk2p; localizes to P-bodies during stationary phase; TPK3 has a paralog, TPK1, that arose from the whole genome duplication"
ATGTATGTTGATCCGATGAACAACAATGAAATCAGGAAATTAAGCATTACTGCCAAGACA
GAAACAACTCCAGATAACGTTGGACAAGACATTCCTGTAAACGCACATTCGGTGCATGAG
GAATGTTCTTCCAACACACCCGTGGAGATAAATGGAAGAAACAGCGGAAAGTTGAAAGAA
GAAGCGTCTGCAGGTATTTGTTTGGTTAAAAAACCAATGCTACAATATAGAGATACCTCA
GGAAAGTATTCCCTAAGTGACTTTCAGATTTTAAGAACTTTGGGAACTGGCTCATTTGGG
AGAGTTCACCTAATTCGTTCCAATCACAATGGGAGGTTTTACGCTTTGAAGACATTGAAA
AAGCACACTATAGTGAAGCTGAAGCAGGTTGAACACACCAATGACGAACGCCGAATGCTT
TCAATTGTTTCACATCCATTCATCATTCGAATGTGGGGAACGTTCCAAGATTCTCAGCAA
GTTTTCATGGTAATGGACTACATTGAAGGTGGTGAATTATTTTCTTTACTACGTAAATCT


---

# Preparation:

**findMM requires samtools and python3 to be in PATH**

In [9]:
!which samtools
!which python

/Users/Jackson/bioinformatics/samtools/bin/samtools
/Users/Jackson/miniconda3/bin/python


In [10]:
!ls

example.ipynb               findMM.py
example_transcriptome.fasta


**findMM help info:**

In [11]:
!./findMM.py -h

usage: findMM.py [-h] [-b <bowtie path>] [-v <INT>] [-p <INT>]
                 [-out <directory>] -i <ebwt_base> -k <INT> -ref <file>

identify multi-mapping transcripts and their connectivity in a reference
transcriptome fasta file

optional arguments:
  -h, --help            show this help message and exit
  -b <bowtie path>      the path to the bowtie executable. Default: bowtie in
                        PATH
  -v <INT>, --mismatches <INT>
                        maximum number of mismatches allowed in alignments
                        (given directly to bowtie parameter -v). Default: 2
  -p <INT>, --threads <INT>
                        Number of alignment threads for bowtie to use (given
                        directly to bowtie parameter -p). Default: 1
  -out <directory>      Directory where script outputs are saved. Default is
                        current directory
  -i <ebwt_base>, --index <ebwt_base>
                        location of bowtie index made from input refe

Arguments in brackets [ ] are optional and/or have default values.

### parameters:
- **b**: in my case, the bowtie executable is in the PATH
- **mismatches**: let's stick with the default of 2
- **threads**: let's set this to 4
- **out**: let's set an output directory to save all of the files to - `./ex_output/`
- **index**: we will need to create a new bowtie index. let's place the index into a folder called `./index/`
- **k**: lets set this to 30
- **reference**: this will be our `./example_transcriptome.fasta` file

### need a bowtie index of the reference transcriptome
Ran the following commands:
> ```bash
> mkdir index
> bowtie-build example_transcriptome.fasta index/example_transcriptome
> ```

In [23]:
!tree

.
├── example.ipynb
├── example_transcriptome.fasta
├── findMM.py
└── index
    ├── example_transcriptome.1.ebwt
    ├── example_transcriptome.2.ebwt
    ├── example_transcriptome.3.ebwt
    ├── example_transcriptome.4.ebwt
    ├── example_transcriptome.rev.1.ebwt
    └── example_transcriptome.rev.2.ebwt

1 directory, 9 files


---

# we should be ready to run findMM.py

In [33]:
!./findMM.py -p 4 -i ./index/example_transcriptome -k 30 -ref example_transcriptome.fasta -out out_test


Running findMM with the following parameters:
     - bowtie path: 'bowtie'
     - number of allowed mismatches (-v): '2'
     - threads used in alignment: '4'
     - output directory: 'out_test'
     - bowtie index: './index/example_transcriptome'
     - k: '30'
     - reference transcriptome: 'example_transcriptome.fasta'
    
OUTPUT FILES:
- k-mer filename:
    out_test/example_transcriptome-30-mers.fa
- multi-mapping (MMing) k-mers:
    out_test/example_transcriptome-30-mers-multi-mapping_kmers.fa
- network csv file:
    out_test/example_transcriptome-30-mers-MM_network_all_connections.csv
- network csv file with duplicate connections removed:
    out_test/example_transcriptome-30-mers-MM_network_unique_connections.csv
- mulit-mapping transcripts table:
    out_test/example_transcriptome-30-mers-multi-mapping_transcripts_table.csv

temperary files:
- k-mer alignment sam (removed):
    out_test/example_transcriptome-30-mers.sam
- MMing k_mer alignment sam (removed):
    out_test/exa

---

# inspecting findMM output

In [1]:
import pandas as pd

In [34]:
!tree

.
├── example.ipynb
├── example_transcriptome.fasta
├── findMM.py
├── index
│   ├── example_transcriptome.1.ebwt
│   ├── example_transcriptome.2.ebwt
│   ├── example_transcriptome.3.ebwt
│   ├── example_transcriptome.4.ebwt
│   ├── example_transcriptome.rev.1.ebwt
│   └── example_transcriptome.rev.2.ebwt
└── out_test
    ├── example_transcriptome-30-mers-MM_network_all_connections.csv
    ├── example_transcriptome-30-mers-MM_network_unique_connections.csv
    ├── example_transcriptome-30-mers-multi-mapping_kmers.fa
    ├── example_transcriptome-30-mers-multi-mapping_transcripts_table.csv
    └── example_transcriptome-30-mers.fa

2 directories, 14 files


file: `out_test/example_transcriptome-30-mers.fa`
<br>\- all of the subsequences of the transcriptome with a length of 'k' <br>

In [41]:
print('transcriptome k-mers:')
!head -n20 out_test/example_transcriptome-30-mers.fa
print('...')

transcriptome k-mers:
>YKL166C.1
ATGTATGTTGATCCGATGAACAACAATGAA
>YKL166C.2
TGTATGTTGATCCGATGAACAACAATGAAA
>YKL166C.3
GTATGTTGATCCGATGAACAACAATGAAAT
>YKL166C.4
TATGTTGATCCGATGAACAACAATGAAATC
>YKL166C.5
ATGTTGATCCGATGAACAACAATGAAATCA
>YKL166C.6
TGTTGATCCGATGAACAACAATGAAATCAG
>YKL166C.7
GTTGATCCGATGAACAACAATGAAATCAGG
>YKL166C.8
TTGATCCGATGAACAACAATGAAATCAGGA
>YKL166C.9
TGATCCGATGAACAACAATGAAATCAGGAA
>YKL166C.10
GATCCGATGAACAACAATGAAATCAGGAAA
...


file: `out_test/example_transcriptome-30-mers-MM_network_all_connections.csv`
<br>\- multi-mapping network <br>

In [42]:
!head -n20 out_test/example_transcriptome-30-mers-MM_network_all_connections.csv

gene,read_origin,MM-kmer-alignments
YKL172W,YKL172W,22
YKL198C,YKL198C,10
YKL201C,YKL201C,468
YKL224C,YLL025W,110
YKL224C,YLL064C,100
YKL224C,YLR037C,124
YKR013W,YKR013W,12
YKR042W,YKR042W,2
YKR045C,YLR177W,1
YKR072C,YKR072C,306
YKR092C,YKR092C,124
YKR092C,YLR177W,54
YKR094C,YLL039C,42
YKR094C,YLR167W,100
YKR102W,YKR102W,3936
YKR103W,YLL048C,1
YLL010C,YLL010C,14
YLL021W,YLL021W,42
YLL025W,YKL224C,110


In [46]:
pd.read_csv('out_test/example_transcriptome-30-mers-multi-mapping_transcripts_table.csv')

,transcript,length (bp),percent of transcript that multimaps,external multimaps,internal multi-maps?
0,YLR157C-C,132,100.00,3.0,no
1,YLR156C-A,132,100.00,3.0,no
2,YLR161W,345,100.00,4.0,no
3,YLR160C,1089,100.00,3.0,no
4,YLR159W,345,100.00,4.0,no
5,YLR159C-A,132,100.00,3.0,no
6,YLR158C,1089,100.00,3.0,no
7,YLR154C-H,132,100.00,3.0,no
8,YLR157W-D,213,100.00,3.0,no
9,YLR155C,1089,100.00,3.0,no


let's compress the k-mer reads because the file size is large

In [47]:
!gzip out_test/example_transcriptome-30-mers-multi-mapping_kmers.fa